In [ ]:
!nvidia-smi

In [ ]:
import zipfile

# Path to the ZIP file you want to unzip
zip_file_path = 'p.zip'
# Directory to extract to
extract_to = 'data'

# Open the zip file in read mode
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents to the specified directory
    zip_ref.extractall(extract_to)

print("File unzipped successfully!")


In [ ]:
!pip install torchvision

In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image
import os

# Load a pre-trained ResNet model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-1])  # Remove the classification layer
model.to(device)
model.eval()

# Preprocessing transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Function to extract features
def extract_features(image_path):
    img = Image.open(image_path).convert('RGB')
    img = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        features = model(img)
    return features.cpu().numpy().flatten()

# Process directory with subfolders
def process_directory_with_subfolders(root_dir):
    features = []
    image_paths = []
    
    for subdir, _, files in os.walk(root_dir):
        for file in files:
            file_path = os.path.join(subdir, file)
            if file.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'tiff')):
                try:
                    features.append(extract_features(file_path))
                    image_paths.append(file_path)
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")
    return torch.tensor(features), image_paths

# Example usage
root_dir = "data"
features, image_paths = process_directory_with_subfolders(root_dir)


In [ ]:
!pip install umap-learn


In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import umap

# Dimensionality reduction
# Option 1: Use t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
features_2d = tsne.fit_transform(features)

# Option 2: Use UMAP
# reducer = umap.UMAP(n_components=2, random_state=42)
# features_2d = reducer.fit_transform(features)

# Plot the clusters
def plot_clusters(features_2d, clusters, image_paths, num_images=100):
    plt.figure(figsize=(12, 8))
    scatter = plt.scatter(
        features_2d[:, 0], features_2d[:, 1], c=clusters, cmap='tab10', s=5, alpha=0.7
    )
    plt.colorbar(scatter, label='Cluster')
    plt.title('Image Clusters')
    plt.xlabel('Component 1')
    plt.ylabel('Component 2')

    # Optionally annotate some points with image paths
    for i in range(min(num_images, len(image_paths))):
        plt.text(
            features_2d[i, 0], features_2d[i, 1], 
            image_paths[i].split('/')[-1][:5], fontsize=8, alpha=0.6
        )

    plt.show()

# Call the function to plot
plot_clusters(features_2d, clusters, image_paths)


In [ ]:
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Clustering
kmeans = KMeans(n_clusters=4, random_state=42)
clusters = kmeans.fit_predict(features)

# Dimensionality reduction for visualization
tsne = TSNE(n_components=2, random_state=42)
features_2d = tsne.fit_transform(features)

# Plot clusters
plot_clusters(features_2d, clusters, image_paths)


In [ ]:
import os
import shutil
import csv

def get_csv_values(csv_file):
    """
    Reads the CSV file and returns a list of values in the first column.
    """
    values = []
    with open(csv_file, newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header if it exists
        for row in reader:
            if row:  # Check if row is not empty
                values.append(row[0])  # Get the first column value
    return values

def move_images_to_new_folder(src_folder, target_folder):
    """
    Move all image files from the source folder to the target folder.
    Assumes the following image file types: .jpg, .jpeg, .png, .gif, .bmp, .tiff
    """
    image_extensions = {'.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff'}
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)  # Create the target directory if it doesn't exist

    for filename in os.listdir(src_folder):
        file_path = os.path.join(src_folder, filename)
        if os.path.isfile(file_path) and os.path.splitext(filename)[1].lower() in image_extensions:
            shutil.move(file_path, os.path.join(target_folder, filename))  # Move image

def process_folders(root_dir, csv_file):
    """
    Process folders and subfolders within root_dir, check against CSV values,
    and move image files to a new folder inside the 'data_2' directory.
    """
    csv_values = get_csv_values(csv_file)

    # Ensure 'data_2' directory exists
    data_2_dir = os.path.join(root_dir, 'data_2')
    if not os.path.exists(data_2_dir):
        os.makedirs(data_2_dir)

    for foldername, subfolders, filenames in os.walk(root_dir):
        for value in csv_values:
            if value.lower() in foldername.lower():  # Check if folder name contains the value
                # Create the new directory inside 'data_2'
                new_folder = os.path.join(data_2_dir, value)
                move_images_to_new_folder(foldername, new_folder)
                print(f"Moved images from '{foldername}' to '{new_folder}'")
                break  # Stop checking other values once a match is found

# Usage example:
root_directory = "data"
csv_file_path = "put.csv"

process_folders(root_directory, csv_file_path)


In [ ]:
import os

def delete_thumbnails_in_data_2(data_2_dir):
    """
    Deletes any image files in the 'data_2' directory (and its subfolders)
    containing the word 'thumb' in the file name.
    """
    for foldername, subfolders, filenames in os.walk(data_2_dir):
        for filename in filenames:
            if 'thumb' in filename.lower():  # Check if 'thumb' is in the filename (case insensitive)
                file_path = os.path.join(foldername, filename)
                os.remove(file_path)
                print(f"Deleted thumbnail: {file_path}")

# Usage example:
data_2_directory = "data/data_2"

# Delete any "thumb" images in 'data_2' and its subdirectories
delete_thumbnails_in_data_2(data_2_directory)


In [ ]:
import os

def count_images_in_data_2(data_2_dir):
    """
    Counts the number of image files in the 'data_2' directory (and its subfolders).
    Assumes the following image file types: .jpg, .jpeg, .png, .gif, .bmp, .tiff
    """
    image_extensions = {'.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff'}
    image_count = 0

    for foldername, subfolders, filenames in os.walk(data_2_dir):
        for filename in filenames:
            if os.path.splitext(filename)[1].lower() in image_extensions:
                image_count += 1

    return image_count

# Usage example:
data_2_directory = "data/data_2"

# Count images in the 'data_2' directory and its subdirectories
total_images = count_images_in_data_2(data_2_directory)
print(f"Total number of images in 'data_2': {total_images}")


In [ ]:
import os
from PIL import Image

def check_image_sizes(data_2_dir):
    """
    Prints out the size (dimensions) of each image in the 'data_2' directory (and its subfolders).
    """
    image_sizes = []

    for foldername, subfolders, filenames in os.walk(data_2_dir):
        for filename in filenames:
            if filename.lower().endswith(('jpg', 'jpeg', 'png', 'gif', 'bmp', 'tiff')):
                file_path = os.path.join(foldername, filename)
                with Image.open(file_path) as img:
                    width, height = img.size
                    image_sizes.append((file_path, width, height))

    return image_sizes

# Usage example:
data_2_directory = "data/data_2"

# Get image sizes
image_sizes = check_image_sizes(data_2_directory)

# Print out the dimensions of the images
for image_path, width, height in image_sizes:
    print(f"Image: {image_path}, Width: {width}, Height: {height}")


In [ ]:
torch.cuda.empty_cache()


In [ ]:
import os
import torch
import torchvision
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision import models
from PIL import Image
from torch.cuda.amp import GradScaler, autocast

# Custom Dataset class to load grayscale images from data_2 directory
class ImageDataset(Dataset):
    def __init__(self, data_2_dir, transform=None):
        self.data_2_dir = data_2_dir
        self.transform = transform
        self.image_paths = []

        # Walk through the directory and collect image paths
        for foldername, subfolders, filenames in os.walk(self.data_2_dir):
            for filename in filenames:
                if filename.lower().endswith(('jpg', 'jpeg', 'png', 'gif', 'bmp', 'tiff')):
                    self.image_paths.append(os.path.join(foldername, filename))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert("L")  # Convert to grayscale (L mode)
        
        if self.transform:
            img = self.transform(img)
        
        return img

# Set up image transformations (for data augmentation and normalization)
transform = transforms.Compose([
    transforms.Resize((960, 768)),  # Resize images to 960x768
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to 3-channel (RGB)
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize (same as ImageNet)
])

# Load the dataset
data_2_directory = "data/data_2"
dataset = ImageDataset(data_2_directory, transform=transform)

# Set up DataLoader with reduced batch size (to prevent OOM error)
batch_size = 1  # Reduced batch size
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Load a pre-trained ResNet model and modify it for our use case
model = models.resnet50(pretrained=True)

# Freeze all layers except the final layer
for param in model.parameters():
    param.requires_grad = False

# Modify the first layer to accept 3-channel images (even though they are grayscale)
model.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

# Modify the final layer to match the number of classes (we'll set it to an arbitrary number for now)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # Example: Change this number based on your actual class count

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Set up loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)  # Only the final layer's parameters will be optimized

# Mixed Precision Training Setup
scaler = GradScaler()

# Gradient Accumulation Settings
accumulation_steps = 4  # Accumulate gradients over 4 mini-batches

# Training loop (we'll just train for a few epochs as a base model)
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total_images = 0
    optimizer.zero_grad()

    for i, inputs in enumerate(dataloader):
        inputs = inputs.to(device)

        # Mixed precision training: Use autocast to cast operations to float16 where possible
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, torch.zeros(inputs.size(0), dtype=torch.long).to(device))  # Dummy labels
        
        # Scales the loss and calls backward() to perform backpropagation
        scaler.scale(loss).backward()

        # Update the weights using the scaler after gradient accumulation steps
        if (i + 1) % accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        # Track the loss and accuracy
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(torch.argmax(outputs, dim=1) == 0).item()  # Dummy accuracy
        total_images += inputs.size(0)

    epoch_loss = running_loss / total_images
    epoch_acc = running_corrects / total_images * 100
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

    # Clear cache to prevent memory fragmentation
    torch.cuda.empty_cache()

print("Base model training complete!")


In [ ]:
import os
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import torch.cuda.amp as amp
from tqdm.notebook import tqdm

class LocalImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (string): Directory with all the images organized in class-specific subdirectories
            transform (callable, optional): Optional transform to be applied on a sample
        """
        self.root_dir = root_dir
        self.transform = transform
        
        # Get class names and create label mapping
        self.classes = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        
        # Collect all image paths and their labels
        self.images = []
        self.labels = []
        
        for cls_name in self.classes:
            class_dir = os.path.join(root_dir, cls_name)
            for img_name in os.listdir(class_dir):
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                    self.images.append(os.path.join(class_dir, img_name))
                    self.labels.append(self.class_to_idx[cls_name])
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

def fine_tune_model(
    dataset_path, 
    batch_size=8, 
    num_epochs=10, 
    learning_rate=0.0001
):
    # Define transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # ResNet default input size
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Create dataset
    dataset = LocalImageDataset(dataset_path, transform=transform)
    
    # Create DataLoader
    dataloader = DataLoader(
        dataset, 
        batch_size=batch_size, 
        shuffle=True, 
        num_workers=4,  # Adjust based on your system
        pin_memory=True
    )
    
    # Get number of classes
    num_classes = len(dataset.classes)
    print(f"Number of classes detected: {num_classes}")
    print("Classes:", dataset.classes)
    
    # Load pre-trained model
    model = models.resnet50(pretrained=True)
    
    # Modify input layer and final classification layer
    model.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    
    # Move to GPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Mixed precision training
    scaler = amp.GradScaler()
    
    # Training loop
    for epoch in tqdm(range(num_epochs), desc="Epochs", position=0):
        model.train()
        total_loss = 0
        correct_predictions = 0
        total_samples = 0
        
        # Progress bar for batches
        batch_progress = tqdm(dataloader, desc=f"Epoch {epoch+1}", position=1, leave=False)
        
        for inputs, labels in batch_progress:
            # Move to GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Zero gradients
            optimizer.zero_grad()
            
            # Mixed precision forward pass
            with amp.autocast(enabled=True):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            
            # Backward pass
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            # Metrics
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)
            
            # Update batch progress bar
            batch_progress.set_postfix({
                'Loss': f"{loss.item():.4f}",
                'Accuracy': f"{correct_predictions / total_samples * 100:.2f}%"
            })
        
        # Print epoch statistics
        epoch_loss = total_loss / len(dataloader)
        epoch_accuracy = correct_predictions / total_samples * 100
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")
    
    # Save the model
    save_path = 'fine_tuned_model.pth'
    torch.save({
        'model_state_dict': model.state_dict(),
        'class_to_idx': dataset.class_to_idx,
        'classes': dataset.classes
    }, save_path)
    print(f"Model saved to {save_path}")
    
    return model

# Example usage


In [ ]:
import os
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
from PIL import Image
import torch.cuda.amp as amp
from tqdm.notebook import tqdm
import numpy as np

class LocalImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        
        # Get class names and create label mapping
        self.classes = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        
        # Collect all image paths and their labels
        self.images = []
        self.labels = []
        
        for cls_name in self.classes:
            class_dir = os.path.join(root_dir, cls_name)
            for img_name in os.listdir(class_dir):
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                    self.images.append(os.path.join(class_dir, img_name))
                    self.labels.append(self.class_to_idx[cls_name])
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

def train_test_split(dataset, test_size=0.2, shuffle=True):
    """
    Split dataset into train and test sets
    
    Args:
    - dataset: The full dataset
    - test_size: Proportion of dataset to include in test split (0.0 to 1.0)
    - shuffle: Whether to shuffle before splitting
    
    Returns:
    - train_dataset, test_dataset
    """
    test_size = int(len(dataset) * test_size)
    train_size = len(dataset) - test_size
    
    return random_split(dataset, [train_size, test_size])

def evaluate_model(model, test_dataloader, device):
    """
    Evaluate the model on test dataset
    
    Args:
    - model: Trained PyTorch model
    - test_dataloader: DataLoader for test dataset
    - device: torch.device to run the evaluation on
    
    Returns:
    - Dictionary of evaluation metrics
    """
    model.eval()
    correct = 0
    total = 0
    class_correct = [0] * len(test_dataloader.dataset.dataset.classes)
    class_total = [0] * len(test_dataloader.dataset.dataset.classes)
    
    confusion_matrix = np.zeros((len(test_dataloader.dataset.dataset.classes), 
                                  len(test_dataloader.dataset.dataset.classes)), 
                                 dtype=int)
    
    with torch.no_grad():
        for inputs, labels in tqdm(test_dataloader, desc="Evaluation", position=0):
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            # Per-class accuracy
            for i in range(len(labels)):
                label = labels[i]
                pred = predicted[i]
                class_correct[label] += (pred == label).item()
                class_total[label] += 1
                
                # Confusion matrix
                confusion_matrix[label][pred] += 1
    
    # Calculate metrics
    overall_accuracy = 100 * correct / total
    
    # Per-class accuracy
    class_accuracies = [100 * class_correct[i] / class_total[i] if class_total[i] > 0 else 0 
                        for i in range(len(class_total))]
    
    return {
        'overall_accuracy': overall_accuracy,
        'class_accuracies': class_accuracies,
        'confusion_matrix': confusion_matrix,
        'class_names': test_dataloader.dataset.dataset.classes
    }

def fine_tune_model(
    dataset_path, 
    batch_size=8, 
    num_epochs=10, 
    learning_rate=0.0001,
    test_size=0.2
):
    # Define transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # ResNet default input size
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Create full dataset
    full_dataset = LocalImageDataset(dataset_path, transform=transform)
    
    # Split dataset
    train_dataset, test_dataset = train_test_split(full_dataset, test_size=test_size)
    
    # Create DataLoaders
    train_dataloader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        shuffle=True, 
        num_workers=4,
        pin_memory=True
    )
    
    test_dataloader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )

    # Get number of classes
    num_classes = len(full_dataset.classes)
    print(f"Number of classes detected: {num_classes}")
    print("Classes:", full_dataset.classes)
    print(f"Total images: {len(full_dataset)}")
    print(f"Training images: {len(train_dataset)}")
    print(f"Test images: {len(test_dataset)}")
    
    # Load pre-trained model
    model = models.resnet50(pretrained=True)
    
    # Modify input layer and final classification layer
    model.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    
    # Move to GPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Mixed precision training
    scaler = amp.GradScaler()
    
    # Training loop
    for epoch in tqdm(range(num_epochs), desc="Epochs", position=0):
        model.train()
        total_loss = 0
        correct_predictions = 0
        total_samples = 0
        
        # Progress bar for batches
        batch_progress = tqdm(train_dataloader, desc=f"Epoch {epoch+1}", position=1, leave=False)
        
        for inputs, labels in batch_progress:
            # Move to GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Zero gradients
            optimizer.zero_grad()
            
            # Mixed precision forward pass
            with amp.autocast(enabled=True):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            
            # Backward pass
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            # Metrics
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)
            
            # Update batch progress bar
            batch_progress.set_postfix({
                'Loss': f"{loss.item():.4f}",
                'Accuracy': f"{correct_predictions / total_samples * 100:.2f}%"
            })
        
        # Print epoch statistics
        epoch_loss = total_loss / len(train_dataloader)
        epoch_accuracy = correct_predictions / total_samples * 100
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")
    
    # Evaluate the model
    eval_results = evaluate_model(model, test_dataloader, device)
    
    # Print evaluation results
    print("\nEvaluation Results:")
    print(f"Overall Accuracy: {eval_results['overall_accuracy']:.2f}%")
    
    print("\nClass-wise Accuracies:")
    for cls, acc in zip(eval_results['class_names'], eval_results['class_accuracies']):
        print(f"{cls}: {acc:.2f}%")
    
    # Save the model with evaluation results
    save_path = 'fine_tuned_model.pth'
    torch.save({
        'model_state_dict': model.state_dict(),
        'class_to_idx': full_dataset.class_to_idx,
        'classes': full_dataset.classes,
        'evaluation_results': eval_results
    }, save_path)
    print(f"\nModel saved to {save_path}")
    
    return model, eval_results, test_dataloader



In [ ]:
eval_results

In [ ]:
if __name__ == "__main__":
    # Replace with the path to your local dataset
    dataset_path = "dataset_final"
    
    # Fine-tune the model
    model = fine_tune_model(
        dataset_path, 
        batch_size=16,  # Adjust based on your GPU memory
        num_epochs=1,
        test_size=0.2
    )

In [8]:
def fine_tune_model(
    dataset_path, 
    batch_size=8, 
    num_epochs=10, 
    learning_rate=0.0001,
    test_size=0.2
):
    # Define transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # ResNet default input size
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Create full dataset
    full_dataset = LocalImageDataset(dataset_path, transform=transform)
    
    # Split dataset
    train_dataset, test_dataset = train_test_split(full_dataset, test_size=test_size)
    
    # Create DataLoaders
    train_dataloader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        shuffle=True, 
        num_workers=4,
        pin_memory=True
    )
    
    test_dataloader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )

    # Get number of classes
    num_classes = len(full_dataset.classes)
    print(f"Number of classes detected: {num_classes}")
    print("Classes:", full_dataset.classes)
    print(f"Total images: {len(full_dataset)}")
    print(f"Training images: {len(train_dataset)}")
    print(f"Test images: {len(test_dataset)}")
    
    # Load pre-trained model
    model = models.resnet50(pretrained=True)
    
    # Modify input layer and final classification layer
    model.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    
    # Move to GPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Mixed precision training
    scaler = amp.GradScaler()

    # Initialize history to collect training/validation metrics
    history = {
        'train_accuracy': [],
        'train_loss': [],
        'val_accuracy': [],
        'val_loss': []
    }
    
    # Training loop
    for epoch in tqdm(range(num_epochs), desc="Epochs", position=0):
        model.train()
        total_loss = 0
        correct_predictions = 0
        total_samples = 0
        
        # Progress bar for batches
        batch_progress = tqdm(train_dataloader, desc=f"Epoch {epoch+1}", position=1, leave=False)
        
        for inputs, labels in batch_progress:
            # Move to GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Zero gradients
            optimizer.zero_grad()
            
            # Mixed precision forward pass
            with amp.autocast(enabled=True):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            
            # Backward pass
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            # Metrics
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)
            
            # Update batch progress bar
            batch_progress.set_postfix({
                'Loss': f"{loss.item():.4f}",
                'Accuracy': f"{correct_predictions / total_samples * 100:.2f}%"
            })
        
        # Print epoch statistics
        epoch_loss = total_loss / len(train_dataloader)
        epoch_accuracy = correct_predictions / total_samples * 100
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")
        
        # Save metrics to history
        history['train_loss'].append(epoch_loss)
        history['train_accuracy'].append(epoch_accuracy)
        
        # Validation phase (only every epoch if needed)
        model.eval()
        val_correct = 0
        val_total = 0
        val_loss = 0
        
        with torch.no_grad():
            for inputs, labels in test_dataloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_accuracy = 100 * val_correct / val_total
        val_loss = val_loss / len(test_dataloader)
        
        # Save validation metrics to history
        history['val_loss'].append(val_loss)
        history['val_accuracy'].append(val_accuracy)
        
        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")
    
    # Evaluate the model
    eval_results = evaluate_model(model, test_dataloader, device)
    
    # Print evaluation results
    print("\nEvaluation Results:")
    print(f"Overall Accuracy: {eval_results['overall_accuracy']:.2f}%")
    
    print("\nClass-wise Accuracies:")
    for cls, acc in zip(eval_results['class_names'], eval_results['class_accuracies']):
        print(f"{cls}: {acc:.2f}%")
    
    # Save the model with evaluation results
    save_path = 'fine_tuned_model.pth'
    torch.save({
        'model_state_dict': model.state_dict(),
        'class_to_idx': full_dataset.class_to_idx,
        'classes': full_dataset.classes,
        'evaluation_results': eval_results
    }, save_path)
    print(f"\nModel saved to {save_path}")
    
    return model, eval_results, history


# Perform extensive evaluation
# Example usage:
model, eval_results= fine_tune_model('dataset_final', num_epochs=10)
extensive_evaluation(model, test_dataloader, device, num_epochs, history)


Number of classes detected: 7
Classes: ['.ipynb_checkpoints', 'Core deformation', 'Core displacement', 'Core split', 'Foreign Object Damage', 'Resin buildup', 'Splice gap']
Total images: 3186
Training images: 2548
Test images: 638


/tmp/ipykernel_859750/3080458671.py:64: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/319 [00:00<?, ?it/s]

/tmp/ipykernel_859750/3080458671.py:93: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(enabled=True):


...

Epoch 9:   0%|          | 0/319 [00:00<?, ?it/s]

Epoch 9/10 - Loss: 0.1829, Accuracy: 94.15%
Validation Loss: 0.8381, Validation Accuracy: 76.96%


Epoch 10:   0%|          | 0/319 [00:00<?, ?it/s]

Epoch 10/10 - Loss: 0.1759, Accuracy: 94.00%
Validation Loss: 0.9532, Validation Accuracy: 76.49%


NameError: name 'evaluate_model' is not defined